<a href="https://colab.research.google.com/github/hailusong/nlp-qa/blob/master/nlp_qa_diy_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Setup
To get the source code of transformer release, do this (assuming the release version is 2.11.0)
```
git fetch -a --tags
git checkout tags/v2.11.0
```

Make sure we have access to file command in Linux

In [1]:
!uname -a
!apt-get install file

Linux 21e96c7b6fa1 4.19.104+ #1 SMP Wed Feb 19 05:26:34 PST 2020 x86_64 x86_64 x86_64 GNU/Linux
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmagic-mgc libmagic1
The following NEW packages will be installed:
  file libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 32 not upgraded.
Need to get 275 kB of archives.
After this operation, 5,297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 file amd64 1:5.32-2ubuntu0.4 [22.1 kB]
Fetched 275 kB in 1s (377 kB/s)
Selecting previously unselected package libmagic-mgc.
(Reading database ... 144467 files and directories currently installed.)
Prepa

In [2]:
try:
    import transformers
except ImportError as e:
    # install huggingface
    !pip install transformers

     |████████████████████████████████| 675kB 3.3MB/s 
     |████████████████████████████████| 1.1MB 17.5MB/s 
     |████████████████████████████████| 3.8MB 16.0MB/s 
     |████████████████████████████████| 890kB 43.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=61bb22395a7908723d542a1c30c042c6ec3fae10f8fff43cb0c05ae3a289f469
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)

1.5.0+cu101
2.11.0


In [4]:
!ls -al
!rm -rf nlp-qa
!git clone --depth 1 https://github.com/hailusong/nlp-qa.git

total 16
drwxr-xr-x 1 root root 4096 May 29 18:19 .
drwxr-xr-x 1 root root 4096 Jun  3 19:39 ..
drwxr-xr-x 1 root root 4096 Jun  2 16:14 .config
drwxr-xr-x 1 root root 4096 May 29 18:19 sample_data
Cloning into 'nlp-qa'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9 (delta 0), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [5]:
!pip install -r nlp-qa/question-answering/requirements.txt

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 204kB 9.8MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=67e45a4174d8b02069294cbbf65e51781aa6ae51528b9e0b6be41c725e86dac7
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval
ERROR: pytorch-lightning 0.7.3 has requirement future>=0.17.1, but you'll have future 0.16.0 which is incompatible.


## Fine-tune on SQuAD 1.1 dataset

### Download dataset

In [6]:
%env SQUAD_DIR=./SQUAD-10-dataset
!rmdir -f $SQUAD_DIR
!mkdir -p $SQUAD_DIR
!wget -O $SQUAD_DIR/train-v1.1.json https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
!wget -O $SQUAD_DIR/dev-v1.1.json https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
!wget -O $SQUAD_DIR/evaluate-v1.1.py https://github.com/allenai/bi-att-flow/blob/master/squad/evaluate-v1.1.py

env: SQUAD_DIR=./SQUAD-10-dataset
rmdir: invalid option -- 'f'
Try 'rmdir --help' for more information.
--2020-06-03 19:45:07--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.109.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘./SQUAD-10-dataset/train-v1.1.json’

./SQUAD-10-dataset/ 100%[===================>]  28.88M  46.2MB/s    in 0.6s    

2020-06-03 19:45:08 (46.2 MB/s) - ‘./SQUAD-10-dataset/train-v1.1.json’ saved [30288272/30288272]

--2020-06-03 19:45:09--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.109.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connec

### Fine-tune
run_squad.py should download following data from net:
- model parameter configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json
- vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt
- weights file https://cdn.huggingface.co/bert-base-uncased-pytorch_model.bin

In [7]:
!file -i $SQUAD_DIR/train-v1.1.json
# !iconv -f utf-8 -t ascii//TRANSLIT < $SQUAD_DIR/train-v1.1.json > $SQUAD_DIR/train-v1.1-ascii.json 
!head -c 50 $SQUAD_DIR/train-v1.1.json

./SQUAD-10-dataset/train-v1.1.json: text/plain; charset=us-ascii
{"data": [{"title": "University_of_Notre_Dame", "p

In [8]:
!python ./nlp-qa/question-answering/run_squad.py \
  --model_type bert \
  --model_name_or_path bert-base-uncased \
  --do_train \
  --do_eval \
  --do_lower_case \
  --train_file $SQUAD_DIR/train-v1.1.json \
  --predict_file $SQUAD_DIR/dev-v1.1.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir /tmp/debug_squad/

2020-06-03 19:45:15.963225: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
06/03/2020 19:45:18 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
06/03/2020 19:45:18 - INFO - filelock -   Lock 140284826297960 acquired on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
06/03/2020 19:45:18 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpn3bw7oit
Downloading: 100% 433/433 [00:00<00:00, 324kB/s]
06/03/2020 19:45:18 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json in cache at /root/.cache/torc